# Finding good and bad features

In this notebook, I demonstrate how to use OpenCV's features to detect corners.

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <cstdlib>
#include <vector>
#include <tuple>
#include <stdexcept>
#include <sstream>
// #include <string>
#include <limits>
#include <opencv2/opencv.hpp>
#include <opencv2/tracking.hpp> // selectROI is part of tracking API

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Create the image

I want to create an homogeneous rectangle over an homogenoeus background. Just like this:

![Test image 1](../test_image1_noise.png)

It's rubbish and that's the point I want to illustrate.

In [5]:
Mat input_image = imread("../test_image1_noise.png", IMREAD_COLOR);
int counter = 1;

if (!input_image.data)
    throw runtime_error("Could not find the image, the program will terminate");

## Select a patch

I am going to display the image and use the mouse to select a rectangular patch in the image.

In [6]:
Rect2d roi_rect = selectROI(input_image);
cout << "ROI position in x: " << roi_rect.x << endl;
cout << "ROI position in y: " << roi_rect.y << endl;
cout << "ROI width: " << roi_rect.width << endl;
cout << "ROI height: " << roi_rect.height << endl;

ROI position in x: 106
ROI position in y: 25
ROI width: 43
ROI height: 31


Display the corresponding patch

In [7]:
cv::Mat patch = input_image(roi_rect);
imshow("patch to detect", patch);
waitKey(1);

Write a function to apply the zero-mean, unit-variance normalisation

$Img_2(x,y) = \frac{Img_1(x,y) - \mathrm{avg}(Img_1)}{\mathrm{stddev}(Img_1)}$

In [8]:
Mat zeroMenUnitVariangeNormalisation(const Mat& anImage)
{
    Scalar average;
    Scalar stddev;

    Mat image;
    anImage.copyTo(image);
    cvtColor(image, image, COLOR_BGR2GRAY);
    image.convertTo(image, CV_32F);

    meanStdDev(image, average, stddev, cv::Mat());
        
    return (image - average) / stddev;
}

The average pixel value of `normalised_patch` is zero, the variance is 1.

In [9]:
Mat normalised_patch = zeroMenUnitVariangeNormalisation(patch);

## Compute the auto-correlation function

Create a new image of the right size

1. For every pixel location,
2. Extract the corresponding region of interest (ROI)
3. Transform it using the zero-mean, unit-variance normalisation
4. Perform a pixel-wise product between the two normalised patches
5. Compute its mean pixel value
6. This is the auto-correlation value for that pixel location
7. The auto-correlation values are between -1 and 1.

In [10]:
Mat autocorrelation_float(input_image.rows,
                    input_image.cols,
                    CV_32FC1, 
                    Scalar(0,0,0));

Mat autocorrelation_uchar(input_image.rows,
                    input_image.cols,
                    CV_8UC3, 
                    Scalar(128,128,128));


// 1. For every pixel location,
for (int j = 0; j < autocorrelation_float.rows - roi_rect.height; ++j)
{
    for (int i = 0; i < autocorrelation_float.cols - roi_rect.width; ++i)
    {
        // 2. Extract the corresponding region of interest (ROI)
        Rect roi(i, j, roi_rect.width, roi_rect.height);
        Mat image_roi = input_image(roi);
        
        // 3. Transform it using the zero-mean, unit-variance normalisation
        Mat normalised_image_roi = zeroMenUnitVariangeNormalisation(image_roi);
        
        // 4. Perform a pixel-wise product between the two normalised patches
        // 5. Compute its mean pixel value
        Scalar average = mean(normalised_image_roi.mul(normalised_patch));
        
        // 6. This is the auto-correlation value for that pixel location
        int x = i + roi_rect.width / 2;
        int y = j + roi_rect.height / 2;
        autocorrelation_float.at<float>(Point(x, y)) = average[0];

        // Display an animation
        int rescaled = floor(255.0 * ((average[0] + 1) / 2.0));
        if (rescaled < 0) rescaled = 0;
        else if (rescaled > 255) rescaled = 255;
        
        autocorrelation_uchar.at<Vec3b>(Point(x, y))[0] = rescaled;
        autocorrelation_uchar.at<Vec3b>(Point(x, y))[1] = rescaled;
        autocorrelation_uchar.at<Vec3b>(Point(x, y))[2] = rescaled;
        
        if (!(i % 5) && !(j % 5))
        {
            Mat display_roi;
            input_image.copyTo(display_roi);
            rectangle(display_roi, Point(i,j), Point(i+roi_rect.width,j+roi_rect.height), Scalar(255, 255, 255), 2, 2, 0);

            Mat vis;
            hconcat(display_roi,autocorrelation_uchar,vis);
            imshow("Visualisation", vis);
            waitKey(1);
        }
    }
}

// 7. The auto-correlation values are between -1 and 1.
double min_value, max_value;
minMaxLoc(autocorrelation_float, &min_value, &max_value);
cout << min_value << " " << max_value << endl;

Mat vis;
hconcat(input_image, autocorrelation_uchar, vis);
imshow("Visualisation", vis);
waitKey(1);

imwrite("../patch.png", patch);

imwrite("../autocorrelation.png", ((autocorrelation_float + 1.0) / 2.0) * 255.0);


-0.999397 1


## Very bad patch

See Blue rectangle 

![Patch 1](../ROI_selector_1.png)

Corresponding autocorrelation

![Corresponding autocorrelation](../autocorrelation_1.png)

Any white pixel in the image above correspond to a match. **There are many.**

## A better patch: A Corner

See Blue rectangle 

![Patch 2](../ROI_selector_2.png)

Corresponding autocorrelation

![Corresponding autocorrelation](../autocorrelation_2.png)

Any white pixel in the image above correspond to a match. **There is only one.**